In [2]:
import langchain
print(langchain.__version__)

0.3.27


In [3]:
!pip install langchain-google-genai
!pip install langsmith

In [4]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
llm_response = llm.invoke("tell me a joke")

llm_response

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--62dbac3d-ef74-4f3b-8c96-4ab33ef19763-0', usage_metadata={'input_tokens': 5, 'output_tokens': 276, 'total_tokens': 281, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 261}})

In [6]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="GoogleChatBot"

In [7]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm_response)

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [8]:
chain= llm| output_parser
chain.invoke("tell me a joke")

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [9]:
from dataclasses import field
from typing import List
from pydantic import BaseModel, Field

class Mobile_review(BaseModel):
  phone_model: str=Field(description = "Name and model of the phone")
  rating: float= Field(description= "overall rating out of 5")
  pros: List[str]=Field(description="list of positive aspects")
  cons: List[str]=Field(description="list of negative aspects")
  summary:str=Field(description="berief summery of the review")

review_text="""
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous, 13 colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm=llm.with_structured_output(Mobile_review)
output =structured_llm.invoke(review_text)
output

Mobile_review(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Excellent camera, especially at night', 'Solid battery life'], cons=['Pricey', 'No charger included', 'Awkward button layout (Bixby button)'], summary='Great phone with a few annoying quirks, worth checking out for an upgrade.')

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("tell me a joke about {topic}")
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about programming', additional_kwargs={}, response_metadata={})])

In [11]:
chain =prompt| llm| output_parser
chain.invoke({"topic": "programming"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# define the prompt
prompt=ChatPromptTemplate.from_template("tell me a joke about {topic}")

#intialize the LLM
llm= ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#compose the chain
chain =prompt|llm|output_parser

#use the chain
result =chain.invoke({"topic": "programming"})
print(result)

Here's a classic:

A programmer's wife tells him, "Go to the store and get a loaf of bread. If they have eggs, get a dozen."

The programmer comes home with 12 loaves of bread.

His wife asks, "Why did you buy 12 loaves of bread?!"

The programmer replies, "They had eggs."


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage


system_message = SystemMessage(content="You are a helpfull assistent that tells a jokes")
human_message = HumanMessage(content="tell me about programing")
llm.invoke([system_message,human_message])

AIMessage(content='Why do programmers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9078005e-ce13-442d-890f-05e80e4fd6f8-0', usage_metadata={'input_tokens': 17, 'output_tokens': 440, 'total_tokens': 457, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 427}})

In [14]:
template =ChatPromptTemplate([("system",'You are a helpfull assistent that tells a jokes '),
                              ("human","tell me about {topic}")])

prompt_value = template.invoke({"topic": "programing"})
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpfull assistent that tells a jokes ', additional_kwargs={}, response_metadata={}), HumanMessage(content='tell me about programing', additional_kwargs={}, response_metadata={})])

In [15]:
llm.invoke(prompt_value)

AIMessage(content='Why do programmers always mix up Halloween and Christmas?\n\nBecause Oct 31 equals Dec 25!\n\n(Get it? Octal 31 is the same as Decimal 25!)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--1dce12ce-a21e-43c6-b18b-c64adfb98016-0', usage_metadata={'input_tokens': 18, 'output_tokens': 324, 'total_tokens': 342, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 283}})

In [16]:
!pip install docx2txt pypdf unstructured
!pip install langchain-community
!pip install langchain-google-genai


In [17]:
from langchain_community.document_loaders import Docx2txtLoader,PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
    )
docx_loader = Docx2txtLoader("//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/FYP-Report.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)
print(f" Split the documnents into {len(splits)} chunks")

1
 Split the documnents into 41 chunks


In [18]:
documents[0]

Document(metadata={'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/FYP-Report.docx'}, page_content="3\n\nMedEase\n\n\n\n\n\n\n\n\n\n\n\nSubmitted By\n\nRashid Ali   MCS-172024\n\nZafar Habib   MCS-172041\n\n\n\nSession 2017 – 2019\n\n\n\nDepartment of Computer Science\n\n\n\nKhwaja Fareed University of Engineering & Information Technology\n\n\t\t\tRahim Yar Khan\n\nProject Detail\n\nType (Nature of project)\n\n[✔ ] Development  \t[  ] Research \t\t[  ] R&D\n\nProject Group Members\n\nSr.#\n\nReg. #\n\nStudent Name\n\nEmail ID\n\n*Signature\n\n(i)\n\nMCS172024\n\nRashid Ali\n\nJamrashid633@gmail.com\n\n\n\n(ii)\n\nMCS172041\n\nZafar Habib\n\nZafar.habib.503@gmail.com\n\n\n\n*The candidates confirm that the work submitted is their own and appropriate credit has been given where reference has been made to work of others\n\n\n\nPlagiarism Free Certificate\n\nThis is to certify that, I am ________________________ S/D/o _______________________, group leader of FYP under regi

In [19]:
splits [5]

Document(metadata={'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/FYP-Report.docx'}, page_content='patient interaction through messaging, sending prescription to pharmacies, confirming appointments, information sharing with other doctors, and patient referrals. Unlike other similar kinds of apps, DAS has unique features such as issuing online prescription to patients, referring patients to a specialist, sending health tips to patients, and effectively, reducing the cost of customer service and providing a vital communication link between doctors and patients.')

In [20]:
#import nltk
#nltk.download('punkt')
#!pip install pypdf

In [21]:
import os
from langchain_community.document_loaders import PyPDFLoader
# 1 . function to load documents from a folder
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if filename.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
      elif filename.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
      else:
         print(f"Unsupported file type: {filename}")
         continue
      documents.extend(loader.load())
    return documents

#Load documents form a folder
folder_path = "//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data"
documents = load_documents(folder_path)

print(f"loaded {len(documents)} documents from {folder_path}")
splits = text_splitter.split_documents(documents)
print(f"Split into {len(splits)} chunks")

loaded 360 documents from //Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data
Split into 847 chunks


In [22]:
#Embedding the documents

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)}document cheunks.")

Created embeddings for 847document cheunks.


In [23]:
!pip install sentence-transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embeddings.embed_documents([split.page_content for split in splits])
document_embeddings[0]

I0000 00:00:1756383105.292052  569754 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


/var/folders/3j/h7c61sfx4qb54tspr5kz4hfr0000gn/T/ipykernel_36119/1974328308.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
E0000 00:00:1756383108.688954  571153 ssl_transport_security_utils.cc:114] Corruption detected.
E0000 00:00:1756383108.688971  571153 ssl_transport_security_utils.cc:71] error:100003fc:SSL routines:OPENSSL_internal:SSLV3_ALERT_BAD_RECORD_MAC
E0000 00:00:1756383108.688972  571153 secure_endpoint.cc:241] Decryption error: TSI_DATA_CORRUPTED


[0.021797677502036095,
 -0.04056418687105179,
 0.040483083575963974,
 -0.06579891592264175,
 -0.0007126233540475368,
 -0.0051056514494121075,
 -0.07255402952432632,
 0.02703327313065529,
 0.016841666772961617,
 0.02528001181781292,
 -0.05672534182667732,
 -0.015836423262953758,
 0.10482825338840485,
 -0.005896067246794701,
 0.0630577951669693,
 0.045774318277835846,
 0.024955863133072853,
 -0.01987946778535843,
 0.008743026293814182,
 -0.09387481212615967,
 0.0394282266497612,
 0.043863698840141296,
 0.00010509647836443037,
 0.03501864895224571,
 0.014838732779026031,
 0.025629645213484764,
 0.024890748783946037,
 0.005843641702085733,
 0.10756491869688034,
 -0.02570352889597416,
 0.016698556020855904,
 0.08862573653459549,
 0.09288352727890015,
 0.06739336252212524,
 0.005536782089620829,
 0.041433755308389664,
 -0.01801905781030655,
 -0.012199196964502335,
 -0.006123844534158707,
 -0.014138171449303627,
 -0.07930922508239746,
 0.004129366017878056,
 0.07268595695495605,
 -0.033493299

## Create and persist Chroma Vector store

In [24]:
!pip install -U langchain-chroma

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Initialize embedding function
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
collection_name = "my_collection"

# Correct parameter name is 'embedding', not 'embedding_function'
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    collection_name=collection_name
)

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


Semantic search

In [26]:
# 5.perform similarity search
query = "What are llms?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevent chunks for the Queary:'{query}'\n")
for i,result in enumerate(search_results,1):
    print(f"result{i}:")
    print(f'Source: {result.metadata.get("source", "unknown")}')
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevent chunks for the Queary:'What are llms?'

result1:
Source: //Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf
Content: Prompt	Engineering	for	LLMs
The	Art	and	Science	of	Building	Large	Language	Model–
Based	Applications
John	Berryman	and	Albert	Ziegler

result2:
Source: //Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf
Content: LLMs.
Prompt	engineering
	comes	in	several	levels	of	sophistication.	The	most	basic
form	makes	use	of	only	a	very	thin	application	layer.	For	instance,	when	you
engage	with	ChatGPT,	you’re	crafting	a	prompt	almost	directly;	the	application



In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k":2})
retriever.invoke("What are llms? ")


[Document(id='116aadcb-ec67-462b-bee7-5828d3020fd4', metadata={'page_label': '2', 'title': 'Prompt Engineering for LLMs', 'page': 1, 'total_pages': 359, 'author': 'John Berryman', 'producer': 'calibre 3.32.0 [https://calibre-ebook.com]', 'creationdate': '2025-08-17T04:15:15+00:00', 'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf', 'creator': 'calibre 3.32.0 [https://calibre-ebook.com]'}, page_content='Prompt\tEngineering\tfor\tLLMs\nThe\tArt\tand\tScience\tof\tBuilding\tLarge\tLanguage\tModel–\nBased\tApplications\nJohn\tBerryman\tand\tAlbert\tZiegler'),
 Document(id='cf5df07c-6a54-49c5-9dcb-f563c49bf49b', metadata={'page_label': '26', 'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf', 'page': 25, 'producer': 'calibre 3.32.

In [28]:
from langchain_core.prompts import ChatPromptTemplate
template =""" Answer the question based only on the following context :
{context}
Question: {question}
Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [29]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain =(
    {"context":retriever, "question":RunnablePassthrough()} | prompt
)
rag_chain.invoke("What are llms? ")

ChatPromptValue(messages=[HumanMessage(content=" Answer the question based only on the following context :\n[Document(id='116aadcb-ec67-462b-bee7-5828d3020fd4', metadata={'author': 'John Berryman', 'total_pages': 359, 'page_label': '2', 'page': 1, 'title': 'Prompt Engineering for LLMs', 'creator': 'calibre 3.32.0 [https://calibre-ebook.com]', 'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf', 'producer': 'calibre 3.32.0 [https://calibre-ebook.com]', 'creationdate': '2025-08-17T04:15:15+00:00'}, page_content='Prompt\\tEngineering\\tfor\\tLLMs\\nThe\\tArt\\tand\\tScience\\tof\\tBuilding\\tLarge\\tLanguage\\tModel–\\nBased\\tApplications\\nJohn\\tBerryman\\tand\\tAlbert\\tZiegler'), Document(id='cf5df07c-6a54-49c5-9dcb-f563c49bf49b', metadata={'total_pages': 359, 'page': 25, 'title': 'Prompt Engineering for LLMs', 'author': 'John Berryman', 'source': '//User

In [30]:
def docs2str(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [31]:
rag_chain =(
    {"context": retriever | docs2str,"question":RunnablePassthrough()}| prompt
)
rag_chain.invoke("What are llms? ")

ChatPromptValue(messages=[HumanMessage(content=' Answer the question based only on the following context :\nPrompt\tEngineering\tfor\tLLMs\nThe\tArt\tand\tScience\tof\tBuilding\tLarge\tLanguage\tModel–\nBased\tApplications\nJohn\tBerryman\tand\tAlbert\tZiegler\n\nLLMs.\nPrompt\tengineering\n\tcomes\tin\tseveral\tlevels\tof\tsophistication.\tThe\tmost\tbasic\nform\tmakes\tuse\tof\tonly\ta\tvery\tthin\tapplication\tlayer.\tFor\tinstance,\twhen\tyou\nengage\twith\tChatGPT,\tyou’re\tcrafting\ta\tprompt\talmost\tdirectly;\tthe\tapplication\nQuestion: What are llms? \nAnswer: ', additional_kwargs={}, response_metadata={})])

In [32]:
rag_chain =(
    {"context":retriever, "question":RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)
question = "what is counting tokens?"
response =rag_chain.invoke(question)
print(response)

I am sorry, but the provided context does not contain information about "counting tokens." It mentions "token" in the context of an LLM computing probabilities for all possible tokens before choosing one through a process called sampling.


## Conversational RAG

In [33]:
from langsmith import traceable
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])


In [34]:
chat_history

[HumanMessage(content='what is counting tokens?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am sorry, but the provided context does not contain information about "counting tokens." It mentions "token" in the context of an LLM computing probabilities for all possible tokens before choosing one through a process called sampling.', additional_kwargs={}, response_metadata={})]

In [35]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt =(
    "Given a chat history and the latest user question "
"which might reference context in the chat history, "
"formulate a standalone question which can be understood "
"without the chat history. Do NOT answer the question, "
"just reformulate it if needed and otherwise return it as is."
)
contextualize_q_system_prompt=ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}"),
    ]
)
#history_aware_retriver = create history_aware_retrever(
#llm retriver,contextualize_q_system_prompt
#)
contextualize_chain =contextualize_q_system_prompt| llm | StrOutputParser()
contextualize_chain.invoke({"input": "what is the cost?", "chat_history": chat_history})


'What is the cost associated with using Large Language Models (LLMs), often measured by tokens?'

In [36]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever= create_history_aware_retriever(
    llm,retriever,contextualize_q_system_prompt
)
history_aware_retriever.invoke({"input": "what is the cost?", "chat_history": chat_history})

[Document(id='afba4bb8-16f1-4754-ba7e-7bed22d903ca', metadata={'page': 48, 'author': 'John Berryman', 'page_label': '49', 'title': 'Prompt Engineering for LLMs', 'producer': 'calibre 3.32.0 [https://calibre-ebook.com]', 'creationdate': '2025-08-17T04:15:15+00:00', 'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf', 'total_pages': 359, 'creator': 'calibre 3.32.0 [https://calibre-ebook.com]'}, page_content='token.\tBut\tif\tyou\tpeel\tback\tone\tmore\tlayer\tof\tthe\tonion\tthat\tis\tthe\tLLM,\tit\tturns\nout\tthat\tactually,\tit\tcomputes\tthe\tprobability\tof\t\nall\tpossible\ttokens\n\tbefore\nchoosing\ta\tsingle\tone.\tThe\tprocess\tunder\tthe\thood\tthat\tchooses\tthe\tactual\ttoken\nis\tcalled\t\nsampling\n\t(see\t\nFigure\t2-11\n).'),
 Document(id='96eba86c-3d4b-4cdf-8878-4a9498c08191', metadata={'producer': 'calibre 3.32.0 [https://calibre-ebook.com]

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt= ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI assistent.use the following context to answer the user's question:"),
    # ("system","tell me joke on programming"),
    ("system","context:{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
]
)
question_answering_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answering_chain)

In [38]:
rag_chain.invoke({"input": "what is prompt?", "chat_history": chat_history})

{'input': 'what is prompt?',
 'chat_history': [HumanMessage(content='what is counting tokens?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am sorry, but the provided context does not contain information about "counting tokens." It mentions "token" in the context of an LLM computing probabilities for all possible tokens before choosing one through a process called sampling.', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='7de5c431-d055-49c3-a249-42c893b24fd8', metadata={'title': 'Prompt Engineering for LLMs', 'source': '//Users/manog/Ds_AI/Project/RAG_Langsmit_FastApi/data/prompt-engineering-for-llms-the-art-and-science-of-building-large-language-modelbased-applications-9781098156152.pdf', 'author': 'John Berryman', 'total_pages': 359, 'creationdate': '2025-08-17T04:15:15+00:00', 'page': 157, 'producer': 'calibre 3.32.0 [https://calibre-ebook.com]', 'page_label': '158', 'creator': 'calibre 3.32.0 [https://calibre-ebook.com]'}, page_conte

## Build multi User chatbot 

In [39]:
pip install sqlite3

I0000 00:00:1756383193.316834  569754 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sqlite3
from datetime import datetime

#def reset_application_logs():
#    conn = get_db_connection()
#   conn.execute('DROP TABLE IF EXISTS application_logs')
#    conn.commit()
#    conn.close()
#    create_application_logs()

# Reset table
#reset_application_logs()

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''
        CREATE TABLE IF NOT EXISTS application_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            session_id TEXT,
            user_query TEXT,
            gpt_response TEXT,
            model TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute(
        'INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
        (session_id, user_query, gpt_response, model)
    )
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute(
        'SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at',
        (session_id,)
    )
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [48]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "what is prompt?"
answer1=rag_chain.invoke({"input":question1,"chat_history": chat_history})["answer"]
insert_application_logs(session_id, question1, answer1, "gemini-2.5-flash")
print(f"human: {question1}")
print(f"AI: {answer1}\n")

[]
human: what is prompt?
AI: Based on the context, a prompt is something you craft or create directly when you engage with applications like ChatGPT. It can be a single element or broken into several smaller elements for more precise constructions.



In [49]:
question2 = "what is counting token?"
answer2=rag_chain.invoke({"input":question2,"chat_history": chat_history})["answer"]
insert_application_logs(session_id, question2, answer2, "gemini-2.5-flash")
print(f"human: {question2}")
print(f"AI: {answer2}\n")

human: what is counting token?
AI: The provided text does not contain information about "counting token." It discusses how an LLM computes the probability of all possible tokens before choosing one through a process called sampling.

